# Clase Práctica #3 (Compilación)

## Preliminares

El objetivo de esta clase es implementar un **generador automático de parsers LL(1)**. Tal generador recibiría como entrada la especificación de una gramática. Siguiendo esta idea, primeramente deberíamos ser capaces de representar de forma clara y cómoda tal "especificación" de la gramática.

A continuación se describen un número de clases que nos serán útiles para construir, de forma concisa, una gramática como un objeto de **Python**.

### Símbolos

Modelaremos los **símbolos** del lenguaje con la clase `Symbol`. Esta clase funcionará como base para la definición de terminales y no terminales. Entre las funcionalidades básicas de los símbolos tenemos que:
- Pueden ser agrupados con el operador `+` para formar oraciones.
- Podemos conocer si representa la cadena especial **epsilon** a través de la propiedad `IsEpsilon` que poseen todas las instancias.
- Podemos acceder a la gramática en la que se definió a través del campo `Grammar` de cada instancia.
- Podemos consultar la notación del símbolo a través del campo `Name` de cada instancia.

Los símbolos no deben ser instanciados directamente (ni sus descendiente) con la aplicación de su constructor. En su lugar, utilizaremos una sintaxis descrita más adelante para definirlos junto a la especificación de la gramática.

In [ ]:
from cmp.pycompiler import Symbol

### No Terminales

Los símbolos **no terminales** los modelaremos con la clase `NonTerminal`. Dicha clase extiende la clase `Symbol` para:
- Añadir noción de las producción que tiene al no terminal como cabecera. Estas pueden ser conocidas a través del campo `productions` de cada instancia.
- Permitir añadir producciones para ese no terminal a través del operador `%=`.
- Incluir propiedades `IsNonTerminal` - `IsTerminal` que devolveran `True` - `False` respectivamente.

Los no terminales no deben ser instanciados directamente con la aplicación de su constructor. En su lugar, se presentan las siguientes facilidades para definir no terminales a partir de una instancia `G` de `Grammar`:
- Para definir un único no terminal:
    ```python
    non_terminal_var = G.NonTerminal('<non-terminal-name>')
    # non_terminal_var    <--- variable en la que guardaremos la referencia al no terminal.
    # <non-terminal-name> <--- nombre concreto del no terminal.
    ```
- Para definir el símbolo distingido:
    ```python
    start_var = G.NonTerminal('<start-name>', True)
    # start_var    <--- variable en la que guardaremos la referencia símbolo distinguido.
    # <start-name> <--- nombre concreto del símbolo distinguido.
    ```
- Para definir múltiples no terminales:
    ```python
    var1, var2, ..., varN = G.NonTerminals('<name1> <name2> ... <nameN>')
    # var1, var2, ..., varN    <--- variables en las que guardaremos las referencias a los no terminales.
    # <name1> <name2> ... <nameN> <--- nombres concretos del no terminales (separados por espacios).
    ```

In [ ]:
from cmp.pycompiler import NonTerminal

### Terminales

Los símbolos **terminales** los modelaremos con la clase `Terminal`. Dicha clase extiende la clase `Symbol` para:
- Incluir propiedades `IsNonTerminal` - `IsTerminal` que devolveran `True` - `False` respectivamente.

Los terminales no deben ser instanciados directamente con la aplicación de su constructor. En su lugar, se presentan las siguientes facilidades para definir no terminales a partir de una instancia `G` de `Grammar`:
- Para definir un único terminal:
    ```python
    terminal_var = G.Terminal('<terminal-name>')
    # terminal_var    <--- variable en la que guardaremos la referencia al terminal.
    # <terminal-name> <--- nombre concreto del terminal.
    ```
- Para definir múltiples terminales:
    ```python
    var1, var2, ..., varN = G.Terminals('<name1> <name2> ... <nameN>')
    # var1, var2, ..., varN    <--- variables en las que guardaremos las referencias a los terminales.
    # <name1> <name2> ... <nameN> <--- nombres concretos del terminales (separados por espacios).
    ```

In [ ]:
from cmp.pycompiler import Terminal

### Fin de Cadena (EOF, *$*)
Modelaremos el símbolo de fin de cadena con la clase `EOF`. Dicha clase extiende la clases `Terminal` para heradar su comportamiento.

La clase `EOF` no deberá ser instanciada directamente con la aplicación de su constructor. En su lugar, una instancia concreta para determinada gramática `G` de `Grammar` se construirá automáticamente y será accesible a través de `G.EOF`.

In [ ]:
from cmp.pycompiler import EOF

### Oraciones y Formas Oracionales

Modelaremos los **oraciones** y **formas oracionales** del lenguaje con la clase `Sentence`. Esta clase funcionará como una colección de terminales y no terminales. Entre las funcionalidades básicas que provee tenemos que nos :
- Permite acceder a los símbolos que componen la oración a través del campo `_symbols` de cada instancia.
- Permite conocer si la oración es completamente vacía a través de la propiedad `IsEpsilon`.
- Permite obtener la concatenación con un símbolo u otra oración aplicando el operador `+`.
- Permite conocer la longitud de la oración (cantidad de símbolos que la componen) utilizando la función *build-in* de python `len(...)`.

Las oraciones pueden ser agrupadas usando el operador `|`. Esto nos será conveniente para definir las producciones las producciones que tengan la misma cabeza (no terminal en la parte izquierda) en una única sentencia. El grupo de oraciones se maneja con la clase `SentenceList`.

No se deben crear instancias de `Sentence` y `SentenceList` directamente con la aplicación de los respectivos constructores. En su lugar, usaremos el operador `+` entre símbolos para formar las oraciones, y el operador `|` entre oraciones para agruparlas.

In [ ]:
from cmp.pycompiler import Sentence, SentenceList

### Epsilon

Modelaremos tanto la **cadena vacía** como el símbolo que la representa: **epsilon** ($\epsilon$), en la misma clase: `Epsilon`. Dicha clase extiende las clases `Terminal` y `Sentence` por lo que ser comporta como ambas. Sobreescribe la implementación del método `IsEpsilon` para indicar que en efecto toda instancia de la clase reprensenta **epsilon**.

La clase `Epsilon` no deberá ser instanciada directamente con la aplicación de su constructor. En su lugar, una instancia concreta para determinada gramática `G` de `Grammar` se construirá automáticamente y será accesible a través de `G.Epsilon`.

In [ ]:
from cmp.pycompiler import Epsilon

### Producciones

Modelaremos las **producciones** con la clase `Production`. Las funcionalidades básicas con que contamos son:
- Poder acceder la cabecera (parte izquierda) y cuerpo (parte derecha) de cada producción a través de los campos `Left` y `Right` respectivamente.
- Consultar si la producción es de la forma $X \rightarrow \epsilon$ a través de la propiedad `IsEpsilon`.
- Desempaquetar la producción en cabecera y cuerpo usando asignaciones: `left, right = production`.

Las producciones no deben ser instanciadas directamente con la aplicación de su constructor. En su lugar, se presentan las siguientes facilidades para formar producciones a partir de una instancia `G` de `Grammar` y un grupo de terminales y no terminales:
- Para definir una producción de la forma $E \rightarrow E + T$:
    ```python
    E %= E + plus + T
    ```
- Para definir múltiples producciones de la misma cabecera en una única sentencia ($E \rightarrow$ $E + T$ | $E - T$ | $T$):
    ```python
    E %= E + plus + T | E + minus + T | T
    ```
- Para usar *epsilon* en una producción (ejemplo $S \rightarrow$ $aS$ | $\epsilon$) haríamos:
    ```python
    S %= S + a | G.Epsilon
    ```

In [ ]:
from cmp.pycompiler import Production

### Gramática

Modelaremos las **gramáticas** con la clase `Grammar`. Las funcionalidades básicas con que contamos son:
- Definir los símbolos _terminales_ y _no terminales_ de la gramática en cuestión a través de los métodos `Terminal` y `Terminals` para los primeros, y `NonTerminal` y `NonTerminals` para los segundos.
- Definir las producciones de la gramática a partir de la aplicación del operador `%=` entre no terminales y oraciones (estas a su vez formadas por la concatenación de símbolos).
- Acceder a **todas** las _producciones_ a través del campo `Productions` de cada instancia.
- Acceder a **todos** los _terminales_ y _no terminales_ a través de los campos `terminals` y `nonTerminals` respectivamente.
- Acceder al _símbolo distinguido_, _epsilon_ y _fin de cadena_ a través de los campos `startSymbol`, `Epsilon` y `EOF` respectivamente.

In [ ]:
from cmp.pycompiler import Grammar

#### Ejemplo: Gramática de HULK

A continuación de muestra cómo especificar la gramática LL(1) para `HULK` vista en conferencias:

$ E \rightarrow$ $TX$  
$ X \rightarrow$ $+TX$ | $-TX$ | $\epsilon$  
$ T \rightarrow$ $FY$  
$ Y \rightarrow$ $*FY$ | $/FY$ | $\epsilon$  
$ F \rightarrow$ $( E )$ | num

In [ ]:
from cmp.utils import pprint, inspect

G = Grammar()
E = G.NonTerminal('E', True)
T,F,X,Y = G.NonTerminals('T F X Y')
plus, minus, star, div, opar, cpar, num = G.Terminals('+ - * / ( ) num')

E %= T + X
X %= plus + T + X | minus + T + X | G.Epsilon
T %= F + Y
Y %= star + F + Y | div + F + Y | G.Epsilon
F %= num | opar + E + cpar

print(G)

## Generador de parsers LL(1)

Pasemos a implementar un generador de parsers LL(1). Para ello será necesario implementar un algoritmo que compute el conjunto _First_ de los símbolos terminales, no terminales y producciones. Además, implementaremos un algoritmo para computar el _Follow_ de todos los no terminales de la gramática.

Una vez podamos calcular dichos conjuntos, construiremos una tabla con $|V_N|$ filas y $|V_t| + 1$ columnas, que representará la unidad de control del parser. La celda $(i,j)$ de la tabla contiene la producción a aplicar si al "tratar" de parsear el _i-ésimo_ no terminal, el cabezal queda apuntando al símbolo asociado a la columna _j_ (es un terminal o _$_). Aplicar una producción se reduce a consumir terminales y tratar de parsear no terminales, según van apareciendo en la parte derecha de la producción.

### Conjunto de símbolos

Resulta conveniente manejar la pertenencia o no de *epsilon* a un conjunto como un caso extremo. Para ello usaremos la clase `ContainerSet` implementada a continuación.
- La clase funciona como un conjunto de símbolos.
- Permite consulta la pertenencia de _epsilon_ al conjunto.
- Las operaciones que modifican el conjunto devuelven si hubo _cambio_ o _no_.
- El conjunto puede ser actualizado con la adición de elementos individuales, `add(...)`, o a partir de otro conjunto,`update(...)` y `hard_update(...)`.
- La actualización _sin epsilon (1)_, _con epsilon (2)_ y de _solo epsilon (3)_, ocurre a través de `update(...)`, `hard_update(...)` y `epsilon_update(...)` respectivamente.

In [ ]:
class ContainerSet:
    def __init__(self, *values, contains_epsilon=False):
        self.set = set(values)
        self.contains_epsilon = contains_epsilon
        
    def add(self, value):
        n = len(self.set)
        self.set.add(value)
        return n != len(self.set)
        
        
    def set_epsilon(self, value=True):
        last = self.contains_epsilon
        self.contains_epsilon = value
        return last != self.contains_epsilon
        
    def update(self, other):
        n = len(self.set)
        self.set.update(other.set)
        return n != len(self.set)
    
    def epsilon_update(self, other):
        return self.set_epsilon(self.contains_epsilon | other.contains_epsilon)
    
    def hard_update(self, other):
        return self.update(other) | self.epsilon_update(other)
    
    def __len__(self):
        return len(self.set) + int(self.contains_epsilon)
    
    def __str__(self):
        return '%s-%s' % (str(self.set), self.contains_epsilon)
    
    def __repr__(self):
        return str(self)
    
    def __iter__(self):
        return iter(self.set)
    
    def __eq__(self, other):
        return isinstance(other, ContainerSet) and self.set == other.set and self.contains_epsilon == other.contains_epsilon

# Simplemente para usar la definión del modulo cmp
from cmp.utils import ContainerSet

### First de una forma oracional

Recordemos que el conjunto *First* de una forma oracional se define como:
- $First(w) = \{ x \in V_t $ | $ w \Rightarrow^* x \alpha, \alpha \in (V_t \cup V_n)^* \}$
    - $\cup$ $\{ \epsilon  \}$, si $w \Rightarrow^* \epsilon$
    - $\cup$ $\{  \}$, en otro caso.
    
Como vimos en conferencia, es posible computar el conjunto _First_ para los _terminales_, _no terminales_ y _producciones_ de la gramática usando un método de punto fijo. Los _firsts_ se inicializan vacíos y de forma incremental se van actualizando con la aplicación de las siguientes reglas:
- Si `X` $\rightarrow$ `W1 | W2 | ... | Wn` entonces por definición, First(X) = $\cup_i$ First($W_i$).
- Si `X` $\rightarrow \epsilon$ entonces $\epsilon \in$ `First(X)`.
- Si `W = xZ` donde `x` es un terminal, entonces trivialmente `First(W) = { x }`.
- Si `W = YZ` donde `Y` es un no-terminal y `Z` una forma oracional, entonces `First(Y)` $\subseteq$ `First(W)`.
- Si `W = YZ` y $\epsilon \in$ `First(Y)` entonces `First(Z)` $\subseteq$ `First(W)`.

Una vez se termine una iteración sin que ocurran cambios se puede dar por terminado el calculo de los _firsts_.

Implementemos el algoritmo para calcular el _first_ de los símbolos y producciones de la gramática en dos fases:
- Con el método `compute_local_first` calcularemos `First(alpha)`, donde `alpha` es una forma oracional, según la versión "actual" del conjunto _firsts_ ya computada.
- Con el método `compute_firsts` calcularemos todos los conjuntos _firsts_. Para ello, realizaremos actualizaciones a los conjuntos iniciales según los resultados de aplicar `compute_local_first` en cada producción.

#### Primera fase

In [ ]:
# Computes First(alpha), given First(Vt) and First(Vn) 
# alpha in (Vt U Vn)*
def compute_local_first(firsts, alpha):
    first_alpha = ContainerSet()
    
    try:
        alpha_is_epsilon = alpha.IsEpsilon
    except:
        alpha_is_epsilon = False
    
    ###################################################
    # alpha == epsilon ? First(alpha) = { epsilon }
    ###################################################
    #                   <CODE_HERE>                   #
    ###################################################
    
    ###################################################
    # alpha = X1 ... XN
    # First(Xi) subconjunto First(alpha)
    # epsilon pertenece a First(X1)...First(Xi) ? First(Xi+1) subconjunto de First(X) y First(alpha)
    # epsilon pertenece a First(X1)...First(XN) ? epsilon pertence a First(X) y al First(alpha)
    ###################################################
    #                   <CODE_HERE>                   #
    ###################################################
    
    # First(alpha)
    return first_alpha

#### Segunda fase

In [ ]:
# Computes First(Vt) U First(Vn) U First(alpha)
# P: X -> alpha
def compute_firsts(G):
    firsts = {}
    change = True
    
    # init First(Vt)
    for terminal in G.terminals:
        firsts[terminal] = ContainerSet(terminal)
        
    # init First(Vn)
    for nonterminal in G.nonTerminals:
        firsts[nonterminal] = ContainerSet()
    
    while change:
        change = False
        
        # P: X -> alpha
        for production in G.Productions:
            X = production.Left
            alpha = production.Right
            
            # get current First(X)
            first_X = firsts[X]
                
            # init First(alpha)
            try:
                first_alpha = firsts[alpha]
            except KeyError:
                first_alpha = firsts[alpha] = ContainerSet()
            
            # CurrentFirst(alpha)???
            local_first = compute_local_first(firsts, alpha)
            
            # update First(X) and First(alpha) from CurrentFirst(alpha)
            change |= first_alpha.hard_update(local_first)
            change |= first_X.hard_update(local_first)
                    
    # First(Vt) + First(Vt) + First(RightSides)
    return firsts

Comprobemos que el algoritmo está bien implementado. Los conjuntos _First_ resultantes deberían ser:

**Terminales**
```
+   :  { + }
-   :  { - }
*   :  { * }
/   :  { / }
(   :  { ( }
)   :  { ) }
num :  { num }
```

**No Terminales**
```
E  :  { num, ( }
T  :  { num, ( }
F  :  { num, ( }
X  :  { +, -, epsilon }
Y  :  { /, *, epsilon }
```

**Producciones**
```
T X     :  { num, ( }
+ T X   :  { + }
- T X   :  { - }
epsilon :  { epsilon }
F Y     :  { num, ( }
* F Y   :  { * }
/ F Y   :  { / }
num     :  { num }
( E )   :  { ( }
```

In [ ]:
from cmp.languages import BasicHulk
hulk = BasicHulk(G)

firsts = compute_firsts(G)
assert firsts == hulk.firsts

### Follows

Recordemos que el conjunto _Follow_ de un _no terminal_ se define como:
- $Follow(X) = \{ a \in V_t \cup \{ \$ \} $ | $ S\$ \Rightarrow^* \alpha X a \beta , \alpha \in (V_t \cup V_n)^* \},  \beta \in (V_t \cup V_n \cup \{ \$ \})^* \}$

Para calcular los _follows_ de los no terminales procederemos de forma similar a como hicimos con los _firsts_. Aplicaremos un método de punto fijo según las reglas:
- `$` pertenece al `Follow(S)`.
- Por definición `epsilon` nunca pertenece al `Follow(X)` para todo `X`.
- Si `X` $\rightarrow$ `WAZ` siendo `W` y `Z` formas oracionales, y `A` un no-terminal cualquiera, entonces `First(Z) - {` $\epsilon$ `}` $\subseteq$ `Follow(A)`.
- Si `X` $\rightarrow$ `WAZ` y $\epsilon \in$ `First(Z)`, entonces `Follow(X)` $\subseteq$ `Follow(A)`.

In [ ]:
from itertools import islice

def compute_follows(G, firsts):
    follows = { }
    change = True
    
    local_firsts = {}
    
    # init Follow(Vn)
    for nonterminal in G.nonTerminals:
        follows[nonterminal] = ContainerSet()
    follows[G.startSymbol] = ContainerSet(G.EOF)
    
    while change:
        change = False
        
        # P: X -> alpha
        for production in G.Productions:
            X = production.Left
            alpha = production.Right
            
            follow_X = follows[X]
            
            ###################################################
            # X -> zeta Y beta
            # First(beta) - { epsilon } subset of Follow(Y)
            # beta ->* epsilon or X -> zeta Y ? Follow(X) subset of Follow(Y)
            ###################################################
            #                   <CODE_HERE>                   #
            ###################################################

    # Follow(Vn)
    return follows

Comprobemos que el algoritmo está bien implementado. Los conjuntos _Follow_ resultantes deberían ser:

```
E :  { ), $ }
T :  { +, ), -, $ }
F :  { /, +, ), *, -, $ }
X :  { ), $ }
Y :  { +, ), -, $ }
```

In [ ]:
follows = compute_follows(G, firsts)
assert follows == hulk.follows

### Tabla LL(1)

Una vez tenemos todos los conjuntos `First` y `Follow` calculados, construiremos una tabla `T`, donde asociaremos a cada par no-terminal `X` / token `t` una producción (a lo sumo). Dicha producción es la única que tiene sentido aplicar si se debe expandir el no-terminal `X` y el token actual es `t`.

Las reglas generales para generar esta tabla son las siguientes:

1. Si `X` $\to$ `W` y `t` $\in V_t$ pertenece al `First(W)` entonces `T[X,t] = X` $\to$ `W`.
2. Si `X` $\to$ `W` con $\epsilon \in$ `First(W)` y `t` pertenece al `Follow(X)` entonces `T[X,t] = X` $\to$ `W`.

Si al aplicar estas reglas, en cada posición `T[X,t]` obtenemos a lo sumo una producción, entonces decimos que una gramática es LL(1). En caso contrario, tenemos al menos un conflicto, pues hay más de una producción que tiene sentido utilizar en algún caso.

In [ ]:
def build_parsing_table(G, firsts, follows):
    # init parsing table
    M = {}
    
    # P: X -> alpha
    for production in G.Productions:
        X = production.Left
        alpha = production.Right
        
        ###################################################
        # working with symbols on First(alpha) ...
        ###################################################
        #                   <CODE_HERE>                   #
        ###################################################    
        
        
        ###################################################
        # working with epsilon...
        ###################################################
        #                   <CODE_HERE>                   #
        ###################################################
    
    # parsing table is ready!!!
    return M            

Comprobemos que el algoritmo está bien implementado.

In [ ]:
M = build_parsing_table(G, firsts, follows)
assert M == hulk.table

### Parsing Descendente No Recursivo

Una vez obtenida la tabla LL(1) podemos escribir un algoritmo de parsing descendente no recursivo. La idea general consiste en emplear una pila de símbolos, donde iremos construyendo la forma oracional que eventualmente derivará en la cadena a reconocer. Si leemos la pila desde el tope hasta el fondo, en todo momento tendremos una forma oracional que debe generar la parte de la cadena no reconocida.

El símbolo en el tope de la pila representa el terminal o no-terminal a analizar. En caso de ser un terminal, debe coincidir con el token analizado. En caso de ser un no-terminal, se consulta la tabla LL(1) y se ejecuta la producción correspondiente, insertando en la pila (en orden inverso) la forma oracional en que deriva el no-terminal extraído.

In [ ]:
def metodo_predictivo_no_recursivo(G, M=None, firsts=None, follows=None):
    
    # checking table...
    if M is None:
        if firsts is None:
            firsts = compute_firsts(G)
        if follows is None:
            follows = compute_follows(G, firsts)
        M = build_parsing_table(G, firsts, follows)
    
    
    # parser construction...
    def parser(w):
        
        ###################################################
        # w ends with $ (G.EOF)
        ###################################################
        # init:
        ### stack =  ????
        ### cursor = ????
        ### output = ????
        ###################################################
        
        # parsing w...
        while True:
            top = stack.pop()
            a = w[cursor]
            
            ###################################################
            #                   <CODE_HERE>                   #
            ###################################################

        # left parse is ready!!!
        return output
    
    # parser is ready!!!
    return parser
    

Comprobemos que el algoritmo está bien implementado. El parse izquierdo para la cadena `"n * n * n + n * n + n + n $"` es:
```
E -> T X
T -> F Y
F -> num
Y -> * F Y
F -> num
Y -> * F Y
F -> num
Y -> e
X -> + T X
T -> F Y
F -> num
Y -> * F Y
F -> num
Y -> e
X -> + T X
T -> F Y
F -> num
Y -> e
X -> + T X
T -> F Y
F -> num
Y -> e
X -> e
```

In [ ]:
parser = metodo_predictivo_no_recursivo(G, M)
left_parse = parser([num, star, num, star, num, plus, num, star, num, plus, num, plus, num, G.EOF])

assert left_parse == [ 
   Production(E, Sentence(T, X)),
   Production(T, Sentence(F, Y)),
   Production(F, Sentence(num)),
   Production(Y, Sentence(star, F, Y)),
   Production(F, Sentence(num)),
   Production(Y, Sentence(star, F, Y)),
   Production(F, Sentence(num)),
   Production(Y, G.Epsilon),
   Production(X, Sentence(plus, T, X)),
   Production(T, Sentence(F, Y)),
   Production(F, Sentence(num)),
   Production(Y, Sentence(star, F, Y)),
   Production(F, Sentence(num)),
   Production(Y, G.Epsilon),
   Production(X, Sentence(plus, T, X)),
   Production(T, Sentence(F, Y)),
   Production(F, Sentence(num)),
   Production(Y, G.Epsilon),
   Production(X, Sentence(plus, T, X)),
   Production(T, Sentence(F, Y)),
   Production(F, Sentence(num)),
   Production(Y, G.Epsilon),
   Production(X, G.Epsilon),
]

## Ejemplos

Probemos el generador de parser implementado con otras gramáticas.

### Gramática 1

In [ ]:
G = Grammar()
S = G.NonTerminal('S', True)
A,B = G.NonTerminals('A B')
a,b = G.Terminals('a b')

S %= A + B
A %= a + A | a
B %= b + B | b

print(G)

In [ ]:
firsts = compute_firsts(G)
pprint(firsts)

# print(inspect(firsts))
assert firsts == {
   a: ContainerSet(a , contains_epsilon=False),
   b: ContainerSet(b , contains_epsilon=False),
   S: ContainerSet(a , contains_epsilon=False),
   A: ContainerSet(a , contains_epsilon=False),
   B: ContainerSet(b , contains_epsilon=False),
   Sentence(A, B): ContainerSet(a , contains_epsilon=False),
   Sentence(a, A): ContainerSet(a , contains_epsilon=False),
   Sentence(a): ContainerSet(a , contains_epsilon=False),
   Sentence(b, B): ContainerSet(b , contains_epsilon=False),
   Sentence(b): ContainerSet(b , contains_epsilon=False) 
}

In [ ]:
follows = compute_follows(G, firsts)
pprint(follows)

# print(inspect(follows))
assert follows == {
   S: ContainerSet(G.EOF , contains_epsilon=False),
   A: ContainerSet(b , contains_epsilon=False),
   B: ContainerSet(G.EOF , contains_epsilon=False) 
}

In [ ]:
M = build_parsing_table(G, firsts, follows)
M

### Gramatica 2

In [ ]:
G = Grammar()
S = G.NonTerminal('S', True)
A,B,C = G.NonTerminals('A B C')
a,b,c,d,f = G.Terminals('a b c d f')

S %= a + A | B + C | f + B + f
A %= a + A | G.Epsilon
B %= b + B | G.Epsilon
C %= c + C | d

print(G)

In [ ]:
firsts = compute_firsts(G)
pprint(firsts)

# print(inspect(firsts))
assert firsts == {
   a: ContainerSet(a , contains_epsilon=False),
   b: ContainerSet(b , contains_epsilon=False),
   c: ContainerSet(c , contains_epsilon=False),
   d: ContainerSet(d , contains_epsilon=False),
   f: ContainerSet(f , contains_epsilon=False),
   S: ContainerSet(d, a, f, c, b , contains_epsilon=False),
   A: ContainerSet(a , contains_epsilon=True),
   B: ContainerSet(b , contains_epsilon=True),
   C: ContainerSet(c, d , contains_epsilon=False),
   Sentence(a, A): ContainerSet(a , contains_epsilon=False),
   Sentence(B, C): ContainerSet(d, c, b , contains_epsilon=False),
   Sentence(f, B, f): ContainerSet(f , contains_epsilon=False),
   G.Epsilon: ContainerSet( contains_epsilon=True),
   Sentence(b, B): ContainerSet(b , contains_epsilon=False),
   Sentence(c, C): ContainerSet(c , contains_epsilon=False),
   Sentence(d): ContainerSet(d , contains_epsilon=False) 
}

In [ ]:
follows = compute_follows(G, firsts)
pprint(follows)

# print(inspect(follows))
assert follows == {
   S: ContainerSet(G.EOF , contains_epsilon=False),
   A: ContainerSet(G.EOF , contains_epsilon=False),
   B: ContainerSet(d, f, c , contains_epsilon=False),
   C: ContainerSet(G.EOF , contains_epsilon=False) 
}

In [ ]:
M = build_parsing_table(G, firsts, follows)
pprint(M)

# print(inspect(M))
assert M == {
   ( S, a, ): [ Production(S, Sentence(a, A)), ],
   ( S, c, ): [ Production(S, Sentence(B, C)), ],
   ( S, b, ): [ Production(S, Sentence(B, C)), ],
   ( S, d, ): [ Production(S, Sentence(B, C)), ],
   ( S, f, ): [ Production(S, Sentence(f, B, f)), ],
   ( A, a, ): [ Production(A, Sentence(a, A)), ],
   ( A, G.EOF, ): [ Production(A, G.Epsilon), ],
   ( B, b, ): [ Production(B, Sentence(b, B)), ],
   ( B, c, ): [ Production(B, G.Epsilon), ],
   ( B, f, ): [ Production(B, G.Epsilon), ],
   ( B, d, ): [ Production(B, G.Epsilon), ],
   ( C, c, ): [ Production(C, Sentence(c, C)), ],
   ( C, d, ): [ Production(C, Sentence(d)), ] 
}

In [ ]:
parser = metodo_predictivo_no_recursivo(G, M)

left_parse = parser([b, b, d, G.EOF])
pprint(left_parse)

# print(inspect(left_parse))
assert left_parse == [ 
   Production(S, Sentence(B, C)),
   Production(B, Sentence(b, B)),
   Production(B, Sentence(b, B)),
   Production(B, G.Epsilon),
   Production(C, Sentence(d)),
]

### Reconstrucción del árbol de derivación y evaluación
- Implemente un algoritmo básico de tokenización para a partir de un `string` obtener la sequencia de tokens correspondiente. Note que los símbolos terminales de la gramática coincide con los "tipos" de tokens, pero no contienen el lexemas.
- Reconstruya el árbol de derivación a partir del parse izquierdo que devuelve el parser.
- Utilice el árbol de derivación para evaluar la expresión. Note que la estructura de la gramática causa que los operadores (+, -, \*, /) asocien hacia la derecha, lo cual conlleva problemas si se evalúa recursivamente sin considerar tal característica.